## Pre-setup

In [1]:
# import the aetest module
from pyats import aetest
import logging
from genie.testbed import load
import argparse
import sys
import pprint

# set debug level DEBUG, INFO, WARNING
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# set pprint
pp = pprint.PrettyPrinter(indent = 2)

## Time for the timestamps

In [2]:
import time

# %Y: full year (4 digits)
# %m: month (2 digits)
# %d: day (2 digits)
# %H: hours
# %M: minutes
# %S: seconds
# Doc: https://docs.python.org/fr/3/library/time.html#time.strftime

# I have to reuse these two lines every time I need the current_time.
t = time.localtime()
current_time = time.strftime("%Y-%m-%d %H:%M:%S", t)

print(current_time)

2020-11-17 17:38:33


## Loading the testbed, connecting to the device

In [3]:
testbed = load('/home/anorsoni/Projets/2020-CAP-Altitude/pyats-check-os/testbed.yaml')

asr903_5 = testbed.devices["ASR903_5"]

asr903_5.connect(init_exec_commands=[],
                 init_config_commands=[],
                 log_stdout=False)

## Getting and parsing XCONNECT info
### show xconnect all

In [4]:
xconnect = asr903_5.parse('show xconnect all')
pp.pprint(xconnect)

{ 'segment_1': { 'ac Po2:110(Eth VLAN)': { 's1': 'UP',
                                           'segment_2': { 'mpls 10.68.120.250:110': { 's2': 'UP',
                                                                                      'st': 'pri',
                                                                                      'xc': 'UP'}}},
                 'ac Po2:112(Eth VLAN)': { 's1': 'UP',
                                           'segment_2': { 'mpls 10.75.1.50:112015': { 's2': 'UP',
                                                                                      'st': 'pri',
                                                                                      'xc': 'UP'}}},
                 'ac Po2:12(Eth VLAN)': { 's1': 'UP',
                                          'segment_2': { 'mpls 10.75.1.50:12015': { 's2': 'UP',
                                                                                    'st': 'pri',
                                               

## Loading the DB

In [6]:
import sqlite3
db_connection = sqlite3.connect('../sqlite/db/checks.db')
db_cursor = db_connection.cursor()

### Functions to insert in the DB

In [20]:
def add_timestamp_db(my_router, test_name, when_tested, current_time):

    # Adding a line in the timestamps table
    # | MyRouter | TestName | WhenTested | Timestamp |
    timestamps_tuple = []
    timestamps_tuple.extend((my_router, test_name, when_tested, current_time))
    db_cursor.execute('''INSERT INTO timestamps VALUES (?,?,?,?)''', timestamps_tuple)

def add_route_summary_db(my_router, vrf, route_source, networks, subnets, current_time):
    # Creating a route_summary_tuple to insert in the route_summary table.
    # | MyRouter | VRF     | Protocol  | Networks | Subnets | TimeStamp |
    route_summary_tuple = []
    route_summary_tuple.extend((my_router, vrf, route_source, networks, subnets, current_time))

    # Inserting a line for each route_source in the database
    db_cursor.execute('''INSERT INTO route_summary VALUES (?,?,?,?,?,?)''', route_summary_tuple)

def add_route_db(my_router, vrf, protocol, route, current_time):

    # Adding a line in the timestamps table
    # | MyRouter | VRF     | Protocol | Route  | Timestamp  |
    route_tuple = []
    route_tuple.extend((my_router, vrf, protocol, route, current_time))
    db_cursor.execute('''INSERT INTO routes VALUES (?,?,?,?,?)''', route_tuple)

## Injecting route_summary information in the database

In [22]:
my_router = asr903_5.name
when_tested = "before"

# Getting the current_time before the test
t = time.localtime()
current_time = time.strftime("%Y-%m-%d %H:%M:%S", t)

# Adding a timestamp for the current test in the DB
add_timestamp_db(my_router, "route_summary", when_tested, current_time)

for vrf in route_summary['vrf']:

    # I will create a line in the DB for each protocol (route_source)
    for route_source in route_summary['vrf'][vrf]['route_source']:

        # Each routing protocol has its own structure. route_source names:
            # connected
            # static
            # isis
            # bgp
            # internal

        # bgp
        if route_source == "bgp":
            for as_number in route_summary['vrf'][vrf]['route_source'][route_source]:
                networks = route_summary['vrf'][vrf]['route_source'][route_source][as_number]['networks']
                subnets = route_summary['vrf'][vrf]['route_source'][route_source][as_number]['subnets']

                add_route_summary_db(my_router, vrf, route_source, networks, subnets, current_time)

        # connected
        if route_source == "connected":
            networks = route_summary['vrf'][vrf]['route_source'][route_source]['networks']
            subnets = route_summary['vrf'][vrf]['route_source'][route_source]['subnets']
                
            add_route_summary_db(my_router, vrf, route_source, networks, subnets, current_time)


        # internal (has no subnets key)
        if route_source == "internal":
            networks = route_summary['vrf'][vrf]['route_source'][route_source]['networks']
            subnets = 0

            add_route_summary_db(my_router, vrf, route_source, networks, subnets, current_time)    

        # isis
        if route_source == "isis":
            for tag in route_summary['vrf'][vrf]['route_source'][route_source]:
                networks = route_summary['vrf'][vrf]['route_source'][route_source][tag]['networks']
                subnets = route_summary['vrf'][vrf]['route_source'][route_source][tag]['subnets']

                add_route_summary_db(my_router, vrf, route_source, networks, subnets, current_time)

        # static
        if route_source == "static":
            networks = route_summary['vrf'][vrf]['route_source'][route_source]['networks']
            subnets = 0

            add_route_summary_db(my_router, vrf, route_source, networks, subnets, current_time)

# Comitting the changes
db_connection.commit()

## Injecting routes information in the database

In [21]:
my_router = asr903_5.name
when_tested = "before"

# Getting the current_time before the test
t = time.localtime()
current_time = time.strftime("%Y-%m-%d %H:%M:%S", t)

# Adding a timestamp for the current test in the DB
add_timestamp_db(my_router, "routes", when_tested, current_time)

for vrf in routes['vrf']:

    # For each address_family (ipv4, ipv6)
    for address_family in routes['vrf'][vrf]['address_family']:

        for route in routes['vrf'][vrf]['address_family'][address_family]['routes']:
            protocol = routes['vrf'][vrf]['address_family'][address_family]['routes'][route]['source_protocol']

            add_route_db(my_router, vrf, protocol, route, current_time)

# Comitting the changes
db_connection.commit()